# Toronto Data - Question 3

## APPROACH
A) I will start with the answer from Question 1 and 2 to build the PostalCode / Borough / Neighbourhood / Lat Lon table.  
B) Then I willl....


### A) Repeat the answer from Question 1 and 2 to build the PostalCode / Borough / Neighbourhood / Lat / Lon table
I will do this all in one cell as it was clearly documented in Q1 and Q2

In [6]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

#Question 1
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url, allow_redirects=True)
open('PostalCodes.html', 'wb').write(r.content)

with open("PostalCodes.html") as fp:
    soup = BeautifulSoup(fp)
rows = soup.table.find_all('tr')

df = pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])
for row in rows[1:]:
    rtext = row.get_text() 
    cols=rtext.splitlines()
    if cols[2] != 'Not assigned':
        if cols[3] == 'Not assigned':
            cols[3] = cols[2]
        df = df.append({'PostalCode' : cols[1] , 'Borough' : cols[2], 'Neighborhood' : cols[3]} , ignore_index=True)
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
pd.options.display.max_colwidth = 120

#Question 2
filename = "http://cocl.us/Geospatial_data"
gd = pd.read_csv(filename)
gd.head()
neighborhoods=df.join(gd.set_index('Postal Code'), on='PostalCode')
neighborhoods.head()

PostalCode      Borough                            Neighborhood   Latitude  \
0        M1B  Scarborough                          Rouge, Malvern  43.806686   
1        M1C  Scarborough  Highland Creek, Rouge Hill, Port Union  43.784535   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3        M1G  Scarborough                                  Woburn  43.770992   
4        M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

## B) Then I will explore the Postal Code districts, and cluster similar ones together, visualising on the map.
The PostalCode districts (which are a group of neighborhoods as in the above table) will be clustered into similar groups. I will try the whole set of Postal Codes

In [4]:
import numpy as np # library to handle data in a vectorized manner

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [7]:
latitude = 43.737746
longitude = -79.289159
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


Creating the function to get nearby venues:

In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now running the code to get the nearby venues for each neighborhood from foursquare: 

In [80]:
CLIENT_ID = 'xx' # your Foursquare ID
CLIENT_SECRET = 'xx' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: xx
CLIENT_SECRET:xx


In [13]:

toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )


Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [14]:
toronto_venues.shape

(2247, 7)

In [ ]:
How many venues in each postal code district? 

In [15]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                                                                                      
Adelaide, King, Richmond                                                                                                                    100   
Agincourt                                                                                                                                     4   
Agincourt North, L'Amoreaux East, Milliken, Steeles East                                                                                      2   
Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown                                 8   
Alderwood, Long Branch                                                                                                                        9   
Bathurst Manor, Downsview North, Wilson Heights                                                                                              19   
Bayview Village                                                                                                                               4   
Bedford Park, Lawrence Manor East                                                                                                            26   
Berczy Park                                                                                                                                  55   
Birch Cliff, Cliffside West                                                                                                                   4   
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe                                                                                 7   
Brockton, Exhibition Place, Parkdale Village                                                                                                 22   
Business Reply Mail Processing Centre 969 Eastern                                                                                            16   
CFB Toronto, Downsview East                                                                                                                   3   
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara                                   14   
Cabbagetown, St. James Town                                                                                                                  46   
Caledonia-Fairbanks                                                                                                                           5   
Canada Post Gateway Processing Centre                                                                                                        10   
Cedarbrae                                                                                                                                     7   
Central Bay Street                                                                                                                           85   
Chinatown, Grange Park, Kensington Market                                                                                                   100   
Christie                                                                                                                                     15   
Church and Wellesley                                                                                                                         91   
Clairlea, Golden Mile, Oakridge                                                                                                               9   
Clarks Corners, Sullivan, Tam O'Shanter                                                                                                       9   
Cliffcrest, Cliffside, Scarborough Village West                                                                                               2   
Clover

How many unique categories? 

In [17]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 277 uniques categories.


## I am now going to analyze each neighborhood

In [28]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Yoga Studio  Accessories Store  Adult Boutique  Afghan Restaurant  Airport  \
0            0                  0               0                  0        0   
1            0                  0               0                  0        0   
2            0                  0               0                  0        0   
3            0                  0               0                  0        0   
4            0                  0               0                  0        0   

   Airport Food Court  Airport Lounge  Airport Service  Airport Terminal  \
0                   0               0                0                 0   
1                   0               0                0                 0   
2                   0               0                0                 0   
3                   0               0                0                 0   
4                   0               0                0                 0   

   American Restaurant  Antique Shop  Aquarium  Argentinian Restaurant  \
0                    0             0         0                       0   
1                    0             0         0                       0   
2                    0             0         0                       0   
3                    0             0         0                       0   
4                    0             0         0                       0   

   Art Gallery  Art Museum  Arts & Crafts Store  Asian Restaurant  \
0            0           0                    0                 0   
1            0           0                    0                 0   
2            0           0                    0                 0   
3            0           0                    0                 0   
4            0           0                    0                 0   

   Athletics & Sports  Auto Garage  Auto Workshop  BBQ Joint  Baby Store  \
0                   0            0              0          0           0   
1                   0            0              0          0           0   
2                   0            0              0          0           0   
3                   0            0              0          0           0   
4                   0            0              0          0           0   

   Bagel Shop  Bakery  Bank  Bar  Baseball Field  Baseball Stadium  \
0           0       0     0    0               0                 0   
1           0       0     0    0               0                 0   
2           0       0     0    1               0                 0   
3           0       0     0    0               0                 0   
4           0       0     0    0               0                 0   

   Basketball Court  Basketball Stadium  Beach  Beer Bar  Beer Store  \
0                 0                   0      0         0           0   
1                 0                   0      0         0           0   
2                 0                   0      0         0           0   
3                 0                   0      0         0           0   
4                 0                   0      0         0           0   

   Belgian Restaurant  Bike Shop  Bistro  Boat or Ferry  Bookstore  Boutique  \
0                   0          0       0              0          0         0   
1                   0          0       0              0          0         0   
2                   0          0       0              0          0         0   
3                   0          0       0              0          0         0   
4                   0          0       0              0          0         0   

   Brazilian Restaurant  Breakfast Spot  Brewery  Bridal Shop  \
0                     0               0        0            0   
1                     0               0        0            0   
2                     0               0        0            0   
3                     0               0        0            0   
4                     0               0        0            0   

   Bubble Tea Sh

Check shape:

In [30]:
toronto_onehot.shape

(2247, 277)

Now group by Neighbourhood

In [32]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  \
0                                                                                                  Adelaide, King, Richmond   
1                                                                                                                 Agincourt   
2                                                                  Agincourt North, L'Amoreaux East, Milliken, Steeles East   
3             Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown   
4                                                                                                    Alderwood, Long Branch   
5                                                                           Bathurst Manor, Downsview North, Wilson Heights   
6                                                                                                           Bayview Village   
7                                                                                         Bedford Park, Lawrence Manor East   
8                                                                                                               Berczy Park   
9                                                                                               Birch Cliff, Cliffside West   
10                                                            Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe   
11                                                                             Brockton, Exhibition Place, Parkdale Village   
12                                                                        Business Reply Mail Processing Centre 969 Eastern   
13                                                                                              CFB Toronto, Downsview East   
14               CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara   
15                                                                                              Cabbagetown, St. James Town   
16                                                                                                      Caledonia-Fairbanks   
17                                                                                    Canada Post Gateway Processing Centre   
18                                                                                                                Cedarbrae   
19                                                                                                       Central Bay Street   
20                                                                                Chinatown, Grange Park, Kensington Market   
21                                                                                                                 Christie   
22                                                                                                     Church and Wellesley   
23                                                                                          Clairlea, Golden Mile, Oakridge   
24                                                                                  Clarks Corners, Sullivan, Tam O'Shanter   
25                                                                          Cliffcrest, Cliffside, Scarborough Village West   
26                                                   Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park   
27                                                                                           Commerce Court, Victoria Hotel   
28                                                                                                               Davisville   
29                                                                                                         Davisville North   
30                                                        Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West   
31                                             

In [34]:
toronto_grouped.shape

(99, 277)

Print each Postal Code group of neighborhoods along with top 5 venues:

In [36]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2  American Restaurant  0.04
3      Thai Restaurant  0.04
4           Steakhouse  0.04


----Agincourt----
            venue  freq
0          Lounge  0.25
1  Breakfast Spot  0.25
2    Skating Rink  0.25
3  Clothing Store  0.25
4     Yoga Studio  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                             venue  freq
0                             Park   0.5
1                       Playground   0.5
2                      Yoga Studio   0.0
3                    Metro Station   0.0
4  Molecular Gastronomy Restaurant   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store  0.25
1             Pharmacy  0.12
2           Beer Store  0.12
3  Fried Chicken Joint  0.12
4       Sandwich Place  0.12


----Alderwood, Long 

                venue  freq
0      Discount Store   0.2
1    Department Store   0.2
2         Bus Station   0.2
3         Coffee Shop   0.2
4  Chinese Restaurant   0.2


----East Toronto----
                venue  freq
0                Park  0.50
1         Coffee Shop  0.25
2   Convenience Store  0.25
3         Yoga Studio  0.00
4  Mexican Restaurant  0.00


----Emery, Humberlea----
                             venue  freq
0                   Baseball Field   0.5
1    Paper / Office Supplies Store   0.5
2                      Yoga Studio   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----Fairview, Henry Farm, Oriole----
                  venue  freq
0        Clothing Store  0.14
1  Fast Food Restaurant  0.09
2           Coffee Shop  0.06
3      Asian Restaurant  0.03
4                Bakery  0.03


----First Canadian Place, Underground city----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.07
2        Hotel  0.06
3   Restaurant  

            venue  freq
0     Coffee Shop  0.09
1      Restaurant  0.05
2            Café  0.05
3           Hotel  0.05
4  Cosmetics Shop  0.03


----Stn A PO Boxes 25 The Esplanade----
          venue  freq
0   Coffee Shop  0.12
1          Café  0.04
2    Restaurant  0.04
3         Hotel  0.03
4  Cocktail Bar  0.03


----Studio District----
                 venue  freq
0                 Café  0.11
1          Coffee Shop  0.08
2   Italian Restaurant  0.05
3               Bakery  0.05
4  American Restaurant  0.05


----The Annex, North Midtown, Yorkville----
            venue  freq
0  Sandwich Place  0.12
1            Café  0.12
2     Coffee Shop  0.12
3     Pizza Place  0.08
4        Pharmacy  0.04


----The Beaches----
                       venue  freq
0          Health Food Store  0.25
1                Coffee Shop  0.25
2                        Pub  0.25
3  Middle Eastern Restaurant  0.00
4        Monument / Landmark  0.00


----The Beaches West, India Bazaar----
               venu

Put into a dataframe:

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [39]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head()

Neighborhood  \
0                                                                                       Adelaide, King, Richmond   
1                                                                                                      Agincourt   
2                                                       Agincourt North, L'Amoreaux East, Milliken, Steeles East   
3  Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown   
4                                                                                         Alderwood, Long Branch   

  1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0           Coffee Shop                  Café   American Restaurant   
1                Lounge          Skating Rink        Clothing Store   
2                  Park            Playground               Dog Run   
3         Grocery Store   Fried Chicken Joint              Pharmacy   
4           Pizza Place           Coffee Shop                   Gym   

  4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
0            Steakhouse       Thai Restaurant      Sushi Restaurant   
1        Breakfast Spot      Doner Restaurant    Dim Sum Restaurant   
2      Department Store          Dessert Shop    Dim Sum Restaurant   
3           Pizza Place            Beer Store  Fast Food Restaurant   
4          Skating Rink              Pharmacy                   Pub   

  7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
0            Restaurant                   Bar                Bakery   
1                 Diner        Discount Store              Dive Bar   
2                 Diner        Discount Store              Dive Bar   
3        Sandwich Place        Discount Store      Department Store   
4        Sandwich Place                  Pool         Women's Store   

  10th Most Common Venue  
0                    Gym  
1                Dog Run  
2       Doner Restaurant  
3           Dessert Shop  
4                  Diner

## Now Run clustering on Postal Code areas: 

In [40]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 0, 0, 3, 0, 0, 0], dtype=int32)

In [70]:
# add clustering labels
#toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#Only take rows where the clustering is not NaN
toronto_merged = toronto_merged[np.isfinite(toronto_merged['Cluster Labels'])]
toronto_merged

PostalCode           Borough  \
0          M1B       Scarborough   
1          M1C       Scarborough   
2          M1E       Scarborough   
3          M1G       Scarborough   
4          M1H       Scarborough   
5          M1J       Scarborough   
6          M1K       Scarborough   
7          M1L       Scarborough   
8          M1M       Scarborough   
9          M1N       Scarborough   
10         M1P       Scarborough   
11         M1R       Scarborough   
12         M1S       Scarborough   
13         M1T       Scarborough   
14         M1V       Scarborough   
15         M1W       Scarborough   
17         M2H        North York   
18         M2J        North York   
19         M2K        North York   
20         M2L        North York   
22         M2N        North York   
23         M2P        North York   
24         M2R        North York   
25         M3A        North York   
26         M3B        North York   
27         M3C        North York   
28         M3H        North York   
29         M3J        North York   
30         M3K        North York   
31         M3L        North York   
32         M3M        North York   
33         M3N        North York   
34         M4A        North York   
35         M4B         East York   
36         M4C         East York   
37         M4E      East Toronto   
38         M4G         East York   
39         M4H         East York   
40         M4J         East York   
41         M4K      East Toronto   
42         M4L      East Toronto   
43         M4M      East Toronto   
44         M4N   Central Toronto   
45         M4P   Central Toronto   
46         M4R   Central Toronto   
47         M4S   Central Toronto   
48         M4T   Central Toronto   
49         M4V   Central Toronto   
50         M4W  Downtown Toronto   
51         M4X  Downtown Toronto   
52         M4Y  Downtown Toronto   
53         M5A  Downtown Toronto   
54         M5B  Downtown Toronto   
55         M5C  Downtown Toronto   
56         M5E  Downtown Toronto   
57         M5G  Downtown Toronto   
58         M5H  Downtown Toronto   
59         M5J  Downtown Toronto   
60         M5K  Downtown Toronto   
61         M5L  Downtown Toronto   
62         M5M        North York   
63         M5N   Central Toronto   
64         M5P   Central Toronto   
65         M5R   Central Toronto   
66         M5S  Downtown Toronto   
67         M5T  Downtown Toronto   
68         M5V  Downtown Toronto   
69         M5W  Downtown Toronto   
70         M5X  Downtown Toronto   
71         M6A        North York   
72         M6B        North York   
73         M6C              York   
74         M6E              York   
75         M6G  Downtown Toronto   
76         M6H      West Toronto   
77         M6J      West Toronto   
78         M6K      West Toronto   
79         M6L        North York   
80         M6M              York   
81         M6N              York   
82         M6P      West Toronto   
83         M6R      West Toronto   
84         M6S      West Toronto   
85         M7A      Queen's Park   
86         M7R       Mississauga   
87         M7Y      East Toronto   
88         M8V         Etobicoke   
89         M8W         Etobicoke   
90         M8X         Etobicoke   
91         M8Y         Etobicoke   
92         M8Z         Etobicoke   
94         M9B         Etobicoke   
95         M9C         Etobicoke   
97         M9M        North York   
98         M9N              York   
99         M9P         Etobicoke   
100        M9R         Etobicoke   
101        M9V         Etobicoke   
102        M9W         Etobicoke   

                                                                                                                Neighborhood  \
0                                                                                                             Rouge, Malvern   
1                                                                                     Highland Creek, Rouge Hill, Port Union   
2                  

In [74]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    try: 
        cluster = int(cluster)
    except:
        _
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Time to examine the cluster

In [75]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


Borough  Cluster Labels       1st Most Common Venue  \
0         Scarborough             0.0        Fast Food Restaurant   
1         Scarborough             0.0                         Bar   
2         Scarborough             0.0         Rental Car Location   
3         Scarborough             0.0                 Coffee Shop   
4         Scarborough             0.0          Athletics & Sports   
6         Scarborough             0.0            Department Store   
7         Scarborough             0.0                    Bus Line   
8         Scarborough             0.0                       Motel   
9         Scarborough             0.0                        Café   
10        Scarborough             0.0           Indian Restaurant   
11        Scarborough             0.0              Breakfast Spot   
12        Scarborough             0.0                      Lounge   
13        Scarborough             0.0                 Pizza Place   
15        Scarborough             0.0        Fast Food Restaurant   
17         North York             0.0                        Pool   
18         North York             0.0              Clothing Store   
22         North York             0.0            Ramen Restaurant   
24         North York             0.0               Grocery Store   
26         North York             0.0        Gym / Fitness Center   
27         North York             0.0            Asian Restaurant   
28         North York             0.0                 Coffee Shop   
29         North York             0.0                         Bar   
31         North York             0.0               Grocery Store   
32         North York             0.0              Baseball Field   
33         North York             0.0        Gym / Fitness Center   
34         North York             0.0           French Restaurant   
35          East York             0.0                 Pizza Place   
36          East York             0.0                 Curling Ice   
37       East Toronto             0.0                 Coffee Shop   
38          East York             0.0         Sporting Goods Shop   
39          East York             0.0           Indian Restaurant   
41       East Toronto             0.0            Greek Restaurant   
42       East Toronto             0.0              Sandwich Place   
43       East Toronto             0.0                        Café   
45    Central Toronto             0.0                         Gym   
46    Central Toronto             0.0                 Coffee Shop   
47    Central Toronto             0.0                 Pizza Place   
49    Central Toronto             0.0                         Pub   
51   Downtown Toronto             0.0                 Coffee Shop   
52   Downtown Toronto             0.0                 Coffee Shop   
53   Downtown Toronto             0.0                 Coffee Shop   
54   Downtown Toronto             0.0              Clothing Store   
55   Downtown Toronto             0.0                 Coffee Shop   
56   Downtown Toronto             0.0                 Coffee Shop   
57   Downtown Toronto             0.0                 Coffee Shop   
58   Downtown Toronto             0.0                 Coffee Shop   
59   Downtown Toronto             0.0                 Coffee Shop   
60   Downtown Toronto             0.0                 Coffee Shop   
61   Downtown Toronto             0.0                 Coffee Shop   
62         North York             0.0            Sushi Restaurant   
63    Central Toronto             0.0                Home Service   
65    Central Toronto             0.0              Sandwich Place   
66   Downtown Toronto             0.0                        Café   
67   Downtown Toronto             0.0                        Café   
68   Downtown Toronto             0.0             Airport Service   
69   Downtown Toronto             0.0                 Coffee Shop   
70   Downtown Toronto             0.0                 Coffee Shop   
71         N

In [76]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


Borough  Cluster Labels       1st Most Common Venue  \
5       Scarborough             1.0  Construction & Landscaping   
48  Central Toronto             1.0                  Playground   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
5             Playground      Doner Restaurant          Dessert Shop   
48         Women's Store      Doner Restaurant          Dessert Shop   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
5     Dim Sum Restaurant                 Diner        Discount Store   
48    Dim Sum Restaurant                 Diner        Discount Store   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
5               Dive Bar               Dog Run          Women's Store  
48              Dive Bar               Dog Run             Donut Shop

In [77]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
19  North York             3.0    Chinese Restaurant                  Café   
94   Etobicoke             3.0                  Bank         Women's Store   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
19   Japanese Restaurant                  Bank         Women's Store   
94            Donut Shop                 Diner        Discount Store   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
19                 Diner        Discount Store              Dive Bar   
94              Dive Bar               Dog Run      Doner Restaurant   

   9th Most Common Venue 10th Most Common Venue  
19               Dog Run       Doner Restaurant  
94             Drugstore           Dessert Shop

In [78]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
20  North York             4.0             Cafeteria      Doner Restaurant   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
20          Dessert Shop    Dim Sum Restaurant                 Diner   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
20        Discount Store              Dive Bar               Dog Run   

   9th Most Common Venue 10th Most Common Venue  
20            Donut Shop         Farmers Market